In [1]:
# Imports
import requests

In [2]:
# Billing Status
from Helper import StakingRewardsAPIClient

client = StakingRewardsAPIClient()
status = client.get_billing_status()
print(status)

{'plan_name': 'startup yearly v0', 'available_credits': 1499997, 'monthly_quota': 1500000, 'subscribed_at': '2025-11-24T07:32:41Z', 'requests_allowed': True, 'billing_date': '2025-12-24T07:32:41Z'}


In [5]:
# Asset Queries
eth = client.get_assets(symbols=["ETH"], limit=1)
print(eth)

DEBUG - Query: 
        {
          assets(where: {symbols: ["ETH"]}, limit: 3) {
            id
            name
            slug
            description
            symbol
          }
        }
        
{'data': {'assets': [{'id': '5d27aa0de03dd80007d1eb49', 'name': 'Ethereum', 'slug': 'ethereum-2-0', 'description': "the world's largest and most decentralized Layer1 blockchain. The network is used for building dApps, holding assets, transacting and communicating without being controlled by a central authority. The Ethereum vision is to build a digital future on a global scale, that is powerful enough to help all of humanity", 'symbol': 'ETH'}]}}
